In [1]:
import numpy as np
import math
from tqdm.notebook import tqdm
from functools import cache
import copy

import heapq



In [2]:
with open("input_day_20.txt") as f:
    text = f.read()


test = False
if test:
    text = r"""
broadcaster -> a, b, c
%a -> b
%b -> c
%c -> inv
&inv -> a
"""

test2 = False
if test2:
    text = r"""
broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output
"""

text = text.strip()
#text = "button -> broadcaster\n" + text


In [3]:
states = {}
# broadcaster : (type, output_locs)
# flip-flop :   (type, output_locs, on_off)
# conjunction :  (type, output_locs, input_names, memories)

for line in text.split("\n"):
    name, outputs = line.split(" -> ")
    output_locs = [_.strip() for _ in outputs.split(",")]
    if name == "broadcaster":
        states["broadcaster"] = ["broadcaster", output_locs]
    elif name[0] == "%":
        states[name[1:]] = ["flip-flop", output_locs, "off"]
    elif name[0] == "&":
        states[name[1:]] = ["conjunction", output_locs, [], []]

# filling in conjunction memories
for name, s in states.items():
    if s[0] == "conjunction":
        output_locs = s[1]
        input_names = []
        memories = []
        for name2, s2 in states.items():
            if name in s2[1]:
                input_names.append(name2)
                memories.append("low")
        states[name] = ["conjunction", output_locs, input_names, memories]

for k, v in states.items():
    print(k, v)

initial_states = copy.deepcopy(states)


bz ['flip-flop', ['rb', 'mf'], 'off']
tn ['flip-flop', ['kb', 'md'], 'off']
broadcaster ['broadcaster', ['nr', 'tn', 'bx', 'nx']]
jp ['flip-flop', ['ps'], 'off']
kc ['conjunction', ['rx'], ['ph', 'vn', 'kt', 'hn'], ['low', 'low', 'low', 'low']]
dh ['flip-flop', ['kb', 'lt'], 'off']
lt ['flip-flop', ['cq', 'kb'], 'off']
ps ['flip-flop', ['mf', 'fh'], 'off']
sr ['flip-flop', ['nh', 'jh'], 'off']
jg ['flip-flop', ['tv'], 'off']
bx ['flip-flop', ['fd', 'jg'], 'off']
kg ['flip-flop', ['fd', 'lg'], 'off']
fh ['flip-flop', ['dp'], 'off']
hv ['flip-flop', ['mf', 'bz'], 'off']
mj ['flip-flop', ['zv'], 'off']
rz ['flip-flop', ['gq', 'mf'], 'off']
tc ['flip-flop', ['td'], 'off']
bl ['flip-flop', ['fd'], 'off']
lg ['flip-flop', ['fd', 'qj'], 'off']
gq ['flip-flop', ['hc', 'mf'], 'off']
kh ['flip-flop', ['ck'], 'off']
td ['flip-flop', ['kb', 'bm'], 'off']
cq ['flip-flop', ['kx', 'kb'], 'off']
zv ['flip-flop', ['tk'], 'off']
nh ['conjunction', ['kh', 'zv', 'tk', 'mj', 'nx', 'qm', 'ph'], ['sr', 'dj',

In [4]:
# [from_name, to_name, low_or_high]

detailed_print = False

pulse_list = []
num_low = 0
num_high = 0
for i in range(1000):
    pulse_list.append(["button", "broadcaster", "low"])

    while len(pulse_list) != 0:

        curr_pulse = pulse_list.pop(0)

        if detailed_print: print(curr_pulse)

        for k,v in states.items():
            if detailed_print: print("    ", k, v)

        from_name, to_name, low_or_high = curr_pulse
        if low_or_high == "low":
            num_low += 1
        if low_or_high == "high":
            num_high += 1
        if not (to_name in states):
            continue
        to_state = states[to_name]

        if to_state[0] == "broadcaster":
            for name2 in to_state[1]:
                pulse_list.append([to_name, name2, low_or_high])

        elif to_state[0] == "flip-flop":
            if low_or_high == "low":
                if to_state[2] == "off":
                    to_state[2] = "on"
                    for name2 in to_state[1]:
                        pulse_list.append([to_name, name2, "high"])
                else:
                    to_state[2] = "off"
                    for name2 in to_state[1]:
                        pulse_list.append([to_name, name2, "low"])
                states[to_name] = to_state

        elif to_state[0] == "conjunction":

            for i in range(len(to_state[2])):
                if to_state[2][i] == from_name:
                    to_state[3][i] = low_or_high
            states[to_name] = to_state

            all_high = True
            for memory in to_state[3]:
                if memory != "high":
                    all_high = False

            if all_high:
                for name2 in to_state[1]:
                    pulse_list.append([to_name, name2, "low"])

            else:
                for name2 in to_state[1]:
                    pulse_list.append([to_name, name2, "high"])

print(num_low, num_high, num_low * num_high)


17521 42456 743871576


In [5]:
# [from_name, to_name, low_or_high]

detailed_print = False

pulse_list = []

# button press nums for ph, vn, kt, hn
required_low_activations = [[], [], [], []]

num_presses = 0
while True:
    num_presses += 1
    if num_presses % 10_000 == 0:
        print(required_low_activations)

    pulse_list.append(["button", "broadcaster", "low"])

    while len(pulse_list) != 0:

        curr_pulse = pulse_list.pop(0)

        if detailed_print: print(curr_pulse)

        for k,v in states.items():
            if detailed_print: print("    ", k, v)

        from_name, to_name, low_or_high = curr_pulse
        if low_or_high == "low":
            num_low += 1
        if low_or_high == "high":
            num_high += 1
        if not (to_name in states):
            continue
        to_state = states[to_name]

        if to_name == 'ph' and low_or_high == "low":
            required_low_activations[0].append(num_presses)
        elif to_name == 'vn' and low_or_high == "low":
            required_low_activations[1].append(num_presses)
        elif to_name == 'kt' and low_or_high == "low":
            required_low_activations[2].append(num_presses)
        elif to_name == 'hn' and low_or_high == "low":
            required_low_activations[3].append(num_presses)

        if to_state[0] == "broadcaster":
            for name2 in to_state[1]:
                pulse_list.append([to_name, name2, low_or_high])

        elif to_state[0] == "flip-flop":
            if low_or_high == "low":
                if to_state[2] == "off":
                    to_state[2] = "on"
                    for name2 in to_state[1]:
                        pulse_list.append([to_name, name2, "high"])
                else:
                    to_state[2] = "off"
                    for name2 in to_state[1]:
                        pulse_list.append([to_name, name2, "low"])
                states[to_name] = to_state

        elif to_state[0] == "conjunction":

            for i in range(len(to_state[2])):
                if to_state[2][i] == from_name:
                    to_state[3][i] = low_or_high
            states[to_name] = to_state

            all_high = True
            for memory in to_state[3]:
                if memory != "high":
                    all_high = False

            if all_high:
                for name2 in to_state[1]:
                    pulse_list.append([to_name, name2, "low"])

            else:
                for name2 in to_state[1]:
                    pulse_list.append([to_name, name2, "high"])


    

[[2907, 6814], [2797, 6594], [3093, 7186], [3021, 7042]]
[[2907, 6814, 10721, 14628, 18535], [2797, 6594, 10391, 14188, 17985], [3093, 7186, 11279, 15372, 19465], [3021, 7042, 11063, 15084, 19105]]
[[2907, 6814, 10721, 14628, 18535, 22442, 26349], [2797, 6594, 10391, 14188, 17985, 21782, 25579, 29376], [3093, 7186, 11279, 15372, 19465, 23558, 27651], [3021, 7042, 11063, 15084, 19105, 23126, 27147]]
[[2907, 6814, 10721, 14628, 18535, 22442, 26349, 30256, 34163, 38070], [2797, 6594, 10391, 14188, 17985, 21782, 25579, 29376, 33173, 36970], [3093, 7186, 11279, 15372, 19465, 23558, 27651, 31744, 35837, 39930], [3021, 7042, 11063, 15084, 19105, 23126, 27147, 31168, 35189, 39210]]
[[2907, 6814, 10721, 14628, 18535, 22442, 26349, 30256, 34163, 38070, 41977, 45884, 49791], [2797, 6594, 10391, 14188, 17985, 21782, 25579, 29376, 33173, 36970, 40767, 44564, 48361], [3093, 7186, 11279, 15372, 19465, 23558, 27651, 31744, 35837, 39930, 44023, 48116], [3021, 7042, 11063, 15084, 19105, 23126, 27147, 31

KeyboardInterrupt: 

In [7]:
for k in required_low_activations:
    print("frequency:", k[-1] - k[-2], k[-2] - k[-3], k[-3] - k[-4])
    f = k[-1] - k[-2]
    print("phase:", k[-1] % f)

print(3907 * 3797 * 4093 * 4021) # don't understand why this is right and the following line is wrong
print(3907 * 3797 * 4093 * 4021 - 1000)

frequency: 3907 3907 3907
phase: 2907
frequency: 3797 3797 3797
phase: 2797
frequency: 4093 4093 4093
phase: 3093
frequency: 4021 4021 4021
phase: 3021
244151741342687
244151741341687


In [ ]:
# 244151741341687 too low
# 244151741341688 too low

In [ ]:
"""# [from_name, to_name, low_or_high]

detailed_print = False

pulse_list = []

rx_pulsed = False

num_presses = 0
while True:
    num_presses += 1
    if num_presses % 10_000 == 0:
        print(num_presses)

    pulse_list.append(["button", "broadcaster", "low"])

    while len(pulse_list) != 0:

        curr_pulse = pulse_list.pop(0)

        if detailed_print: print(curr_pulse)

        for k,v in states.items():
            if detailed_print: print("    ", k, v)

        from_name, to_name, low_or_high = curr_pulse
        if low_or_high == "low":
            num_low += 1
        if low_or_high == "high":
            num_high += 1
        if not (to_name in states):
            continue
        to_state = states[to_name]

        if to_name == 'rx' and low_or_high == "low":
            rx_pulsed = True
            break

        if to_state[0] == "broadcaster":
            for name2 in to_state[1]:
                pulse_list.append([to_name, name2, low_or_high])

        elif to_state[0] == "flip-flop":
            if low_or_high == "low":
                if to_state[2] == "off":
                    to_state[2] = "on"
                    for name2 in to_state[1]:
                        pulse_list.append([to_name, name2, "high"])
                else:
                    to_state[2] = "off"
                    for name2 in to_state[1]:
                        pulse_list.append([to_name, name2, "low"])
                states[to_name] = to_state

        elif to_state[0] == "conjunction":

            for i in range(len(to_state[2])):
                if to_state[2][i] == from_name:
                    to_state[3][i] = low_or_high
            states[to_name] = to_state

            all_high = True
            for memory in to_state[3]:
                if memory != "high":
                    all_high = False

            if all_high:
                for name2 in to_state[1]:
                    pulse_list.append([to_name, name2, "low"])

            else:
                for name2 in to_state[1]:
                    pulse_list.append([to_name, name2, "high"])

    if rx_pulsed:
        print(i+1)
        break
"""
    


10000
20000
30000
40000
50000
60000


KeyboardInterrupt: 

In [ ]:
# 634415985 too low

# 542848530, must also be too low